```sql
-- COPYRIGHT (C) 2022 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUT2.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Flow Diagrams](#flow-diagrams)**
2. **[Dependencies](#dependencies)**
3. **[Input Layer](#input-layer)**
4. **[Intermediate Layer](#intermediate-layer)**
5. **[Static Layer](#static-layer)**


# **FLOW DIAGRAMS**

---

[![](https://mermaid.ink/img/pako:eNplT8sKwjAQ_JWwpwrqB_QgKHoQPNljI7I2axtsk7DdKCr-u_EFintaZmZnZq9QeUOQw771p6pBFqVWa-1Umj7uasbQqKULUdQKz8QvpuCsnF4i06z1u81AjUZKQ_AhtiiWtgYFx1V_1JCYiSrKrBesrasHm9c9OfMXIcQdGYtCP0lPh2WZ2S_Blw0MoUs4WpN-uD5gDdJQRxrytBrkgwbtbkkXQ-pFC2PFM-R7bHsaAkbxxdlVkAtH-ojmFlOt7q263QHxt2D6)](https://mermaid-js.github.io/mermaid-live-editor/edit#pako:eNplT8sKwjAQ_JWwpwrqB_QgKHoQPNljI7I2axtsk7DdKCr-u_EFintaZmZnZq9QeUOQw771p6pBFqVWa-1Umj7uasbQqKULUdQKz8QvpuCsnF4i06z1u81AjUZKQ_AhtiiWtgYFx1V_1JCYiSrKrBesrasHm9c9OfMXIcQdGYtCP0lPh2WZ2S_Blw0MoUs4WpN-uD5gDdJQRxrytBrkgwbtbkkXQ-pFC2PFM-R7bHsaAkbxxdlVkAtH-ojmFlOt7q263QHxt2D6)

Required steps:

1. **[AzureBlob]()**: contains the static `"population"` data collected by VWS to be used for further processing.
2. **[Staging](#input-layer)**: inserts the raw `Source` data within the `[VWSSTAGE].[VWS_POPULATION]` table.
3. **[Intermediate](#static-layer)**: inserts the `Staging` data and cast the correct `Database Types` within `[VWSINTER].[VWS_POPULATION]` using business logic implemented in `[dbo].[SP_INSERT_IL_VWS_POPULATION]`.

- **<font color=teal>IL</font>**: Intermediate Layer
- **<font color=teal>SL</font>**: Static Layer

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/MunicipalityRemapping.ipynb"
        // Additional dependencies (!NOTE! DO NOT FORGET THE COMMA (i.e. ,))
    ]
}
```

# **INPUT LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTAGE].[VWS_POPULATION]'))
CREATE TABLE [VWSSTAGE].[VWS_POPULATION] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[GM_CODE]  [VARCHAR](100) NULL,
	[GM_NAME] [VARCHAR](100) NULL,
	[AGE] [VARCHAR](100) NULL,
	[SEX] [VARCHAR](100) NULL,
	[DATE_OF_POLL] [VARCHAR](100) NULL,
	[POPULATION] [VARCHAR](100) NULL,
	[VR_CODE] [VARCHAR](100) NULL,
	[VR_NAME] [VARCHAR](100) NULL,
	[PROVINCE_CODE] [VARCHAR](100) NULL,
	[PROVINCE_NAME] [VARCHAR](100) NULL,
	[GGD_CODE] [VARCHAR](100) NULL,
	[GGD_NAME] [VARCHAR](100) NULL
);
GO

# **INTERMEDIATE LAYER**

---

## **<span style='color:teal'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSINTER].[VWS_POPULATION]'))
CREATE TABLE [VWSINTER].[VWS_POPULATION] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[GM_CODE]  [VARCHAR](100) NULL,
	[GM_NAME] [VARCHAR](100) NULL,
	[AGE] [VARCHAR](100) NULL,
	[SEX] [VARCHAR](100) NULL,
	[DATE_OF_POLL] [VARCHAR](100) NULL,
	[POPULATION] [VARCHAR](100) NULL,
	[VR_CODE] [VARCHAR](100) NULL,
	[VR_NAME] [VARCHAR](100) NULL,
	[PROVINCE_CODE] [VARCHAR](100) NULL,
	[PROVINCE_NAME] [VARCHAR](100) NULL,
	[GGD_CODE] [VARCHAR](100) NULL,
	[GGD_NAME] [VARCHAR](100) NULL
);
GO

## **<span style='color:teal'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_VWS_POPULATION_DATA]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[VWS_POPULATION] (
        [GM_CODE],
        [GM_NAME],
        [AGE],
        [SEX],
        [DATE_OF_POLL],
        [POPULATION],
        [VR_CODE],
        [VR_NAME],
        [PROVINCE_CODE],
        [PROVINCE_NAME],
        [GGD_CODE],
        [GGD_NAME]
    )
    SELECT
        ISNULL(T2.[GM_CODE_NEW],[GM_CODE]),
        ISNULL(T2.[GM_NAME_NEW],[GM_NAME]),
        T1.[AGE],
        T1.[SEX],
        CAST(T1.[DATE_OF_POLL] AS DATE),
        ROUND(SUM(CAST(T1.[POPULATION] AS INT) * ISNULL(T2.[SHARES],1.0)),0), 
        ISNULL(T2.[VR_CODE],T1.[VR_CODE]),
        ISNULL(T2.[VR_NAME],T1.[VR_NAME]),
        ISNULL(T2.[PROVINCE_CODE],T1.[PROVINCE_CODE]),
        ISNULL(T2.[PROVINCE_NAME],T1.[PROVINCE_NAME]),
        ISNULL(T2.[GGD_CODE], T1.[GGD_CODE]),
        ISNULL(T2.[GGD_NAME],T1.[GGD_NAME])
    FROM
        [VWSSTAGE].[VWS_POPULATION] T1
    LEFT JOIN (
        SELECT 
            * 
        FROM [VWSSTATIC].[VWS_MUNICIPALITY_REMAPPING] WITH(NOLOCK)
        WHERE [ACTIVE] = 1 
            AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[VWS_MUNICIPALITY_REMAPPING] WITH(NOLOCK))  
    ) T2 ON T1.[GM_CODE] = T2.[GM_CODE_ORIGINAL]
    WHERE T1.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[VWS_POPULATION])
    GROUP BY
        ISNULL(T2.[GM_CODE_NEW],[GM_CODE]),
        ISNULL(T2.[GM_NAME_NEW],[GM_NAME]),
        T1.[AGE],
        T1.[SEX],
        CAST(T1.[DATE_OF_POLL] AS DATE),
        ISNULL(T2.[VR_CODE],T1.[VR_CODE]), 
        ISNULL(T2.[VR_NAME],T1.[VR_NAME]), 
        ISNULL(T2.[PROVINCE_CODE],T1.[PROVINCE_CODE]),
        ISNULL(T2.[PROVINCE_NAME],T1.[PROVINCE_NAME]),
        ISNULL(T2.[GGD_CODE],T1.[GGD_CODE]),
        ISNULL(T2.[GGD_NAME],T1.[GGD_NAME]) 
    ORDER BY 
        CAST(T1.[DATE_OF_POLL] AS DATE), 
        ISNULL(T2.[GM_CODE_NEW],[GM_CODE])
END;
GO

# **STATIC LAYER**

---

## **<span style='color:teal'>POPULATION PER AGE 10</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTATIC].[VWS_POPULATION_PER_AGE_GROUP_10]'))
CREATE TABLE [VWSSTATIC].[VWS_POPULATION_PER_AGE_GROUP_10] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	[AGE_GROUP] [VARCHAR](10) NULL,
	[DATE_OF_POLL] [DATE] NULL,
	[POPULATION] [INT] NULL
);
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_VWS_POPULATION_PER_AGE_GROUP_10]
AS
BEGIN
    WITH CTE AS (
        SELECT 
            [DATE_OF_POLL],
            [AGE],
            CASE 
                WHEN [AGE] >= 0 AND [AGE] <= 9 THEN '0-9'
                WHEN [AGE] >= 10 AND [AGE] <= 19 THEN '10-19'
                WHEN [AGE] >= 20 AND [AGE] <= 29 THEN '20-29'
                WHEN [AGE] >= 30 AND [AGE] <= 39 THEN '30-39'
                WHEN [AGE] >= 40 AND [AGE] <= 49 THEN '40-49'
                WHEN [AGE] >= 50 AND [AGE] <= 59 THEN '50-59'
                WHEN [AGE] >= 60 AND [AGE] <= 69 THEN '60-69'
                WHEN [AGE] >= 70 AND [AGE] <= 79 THEN '70-79'
                WHEN [AGE] >= 80 AND [AGE] <= 89 THEN '80-89'
                WHEN [AGE] >= 90 THEN '90+'
                ELSE NULL
            END AS [AGE_GROUP],
            [POPULATION]
        FROM 
            [VWSINTER].[VWS_POPULATION]
        WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED])  FROM [VWSINTER].[VWS_POPULATION])
    )
    INSERT INTO [VWSSTATIC].[VWS_POPULATION] (
        [DATE_OF_POLL],
        [AGE_GROUP],
        [POPULATION]
    )
    SELECT
        [DATE_OF_POLL],
        [AGE_GROUP],
        SUM([POPULATION])
    FROM
        CTE
    WHERE [AGE_GROUP] IS NOT NULL
    GROUP BY [AGE_GROUP], [DATE_OF_POLL]
    ORDER BY [DATE_OF_POLL], [AGE_GROUP]
END;
GO

## **<span style='color:teal'>POPULATION PER AGE DECEASED</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTATIC].[VWS_POPULATION_PER_AGE_GROUP_DECEASED]'))
CREATE TABLE [VWSSTATIC].[VWS_POPULATION_PER_AGE_GROUP_DECEASED] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	-- ADD COLUMNS
);
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_VWS_POPULATION_DATA]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[VWS_POPULATION] (
        -- ADD COLUMN NAMES
    )
    SELECT
        -- SELECT COLUMNS 
    FROM
        [VWSSTAGE].[VWS_POPULATION]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[VWS_POPULATION])
END;
GO

## **<span style='color:teal'>POPULATION PER AGE HOSPITAL IC</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTATIC].[VWS_POPULATION]'))
CREATE TABLE [VWSSTATIC].[VWS_POPULATION] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	-- ADD COLUMNS
);
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_VWS_POPULATION_DATA]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[VWS_POPULATION] (
        -- ADD COLUMN NAMES
    )
    SELECT
        -- SELECT COLUMNS 
    FROM
        [VWSSTAGE].[VWS_POPULATION]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[VWS_POPULATION])
END;
GO

## **<span style='color:teal'>POPULATION NL</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTATIC].[VWS_POPULATION]'))
CREATE TABLE [VWSSTATIC].[VWS_POPULATION] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	-- ADD COLUMNS
);
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_VWS_POPULATION_DATA]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[VWS_POPULATION] (
        -- ADD COLUMN NAMES
    )
    SELECT
        -- SELECT COLUMNS 
    FROM
        [VWSSTAGE].[VWS_POPULATION]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[VWS_POPULATION])
END;
GO

## **<span style='color:teal'>POPULATION PER GM</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTATIC].[VWS_POPULATION]'))
CREATE TABLE [VWSSTATIC].[VWS_POPULATION] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	-- ADD COLUMNS
);
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_VWS_POPULATION_DATA]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[VWS_POPULATION] (
        -- ADD COLUMN NAMES
    )
    SELECT
        -- SELECT COLUMNS 
    FROM
        [VWSSTAGE].[VWS_POPULATION]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[VWS_POPULATION])
END;
GO

## **<span style='color:teal'>POPULATION PER VR</span>**

### **<span style='color:cadetblue'>TABLES</span>**

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS (SELECT * FROM [SYS].[TABLES] WHERE [OBJECT_ID] = OBJECT_ID('[VWSSTATIC].[VWS_POPULATION]'))
CREATE TABLE [VWSSTATIC].[VWS_POPULATION] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
	-- ADD COLUMNS
);
GO

### **<span style='color:cadetblue'>STORED PROCEDURES</span>**

In [ ]:
-- 1) CREATE STORED PROCEDURE(S): STAGING TO STATIC.....
CREATE OR ALTER PROCEDURE [dbo].[SP_INSERT_SL_VWS_POPULATION_DATA]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[VWS_POPULATION] (
        -- ADD COLUMN NAMES
    )
    SELECT
        -- SELECT COLUMNS 
    FROM
        [VWSSTAGE].[VWS_POPULATION]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[VWS_POPULATION])
END;
GO